In [113]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix, accuracy_score

from pystruct.learners import OneSlackSSVM
from pystruct.datasets import load_snakes
from pystruct.utils import make_grid_edges, edge_list_to_features
from pystruct.models import EdgeFeatureGraphCRF


def one_hot_colors(x):
    x = x / 255
    flat = np.dot(x.reshape(-1, 3), 2 ** np.arange(3))
    one_hot = label_binarize(flat, classes=[1, 2, 3, 4, 6])
    return one_hot.reshape(x.shape[0], x.shape[1], 5)


def neighborhood_feature(x):
    """Add a 3x3 neighborhood around each pixel as a feature."""
    # we could also use a four neighborhood, that would work even better
    # but one might argue then we are using domain knowledge ;)
    features = np.zeros((x.shape[0], x.shape[1], 5, 9))
    # position 3 is background.
    features[:, :, 3, :] = 1
    features[1:, 1:, :, 0] = x[:-1, :-1, :]
    features[:, 1:, :, 1] = x[:, :-1, :]
    features[:-1, 1:, :, 2] = x[1:, :-1, :]
    features[1:, :, :, 3] = x[:-1, :, :]
    features[:-1, :-1, :, 4] = x[1:, 1:, :]
    features[:-1, :, :, 5] = x[1:, :, :]
    features[1:, :-1, :, 6] = x[:-1, 1:, :]
    features[:, :-1, :, 7] = x[:, 1:, :]
    features[:, :, :, 8] = x[:, :, :]
    return features.reshape(x.shape[0] * x.shape[1], -1)


def prepare_data(X):
    X_directions = []
    X_edge_features = []
    for x in X:
        # get edges in grid
        right, down = make_grid_edges(x, return_lists=True)
        edges = np.vstack([right, down])
        # use 3x3 patch around each point
        features = neighborhood_feature(x)
        # simple edge feature that encodes just if an edge is horizontal or
        # vertical
        edge_features_directions = edge_list_to_features([right, down])
        # edge feature that contains features from the nodes that the edge connects
        edge_features = np.zeros((edges.shape[0], features.shape[1], 2))
        edge_features[:len(right), :, 0] = features[right[:, 0]]
        edge_features[:len(right), :, 1] = features[right[:, 1]]
        edge_features[len(right):, :, 0] = features[down[:, 0]]
        edge_features[len(right):, :, 1] = features[down[:, 1]]
        edge_features = edge_features.reshape(edges.shape[0], -1)
        X_directions.append((features, edges, edge_features_directions))
        X_edge_features.append((features, edges, edge_features))
    return X_directions, X_edge_features


print("Please be patient. Learning will take 5-20 minutes.")
snakes = load_snakes()
X_train, Y_train = snakes['X_train'], snakes['Y_train']

X_train = [one_hot_colors(x) for x in X_train]
Y_train_flat = [y_.ravel() for y_ in Y_train]

X_train_directions, X_train_edge_features = prepare_data(X_train)

inference = 'qpbo'

X_test, Y_test = snakes['X_test'], snakes['Y_test']
X_test = [one_hot_colors(x) for x in X_test]
Y_test_flat = [y_.ravel() for y_ in Y_test]
X_test_directions, X_test_edge_features = prepare_data(X_test)

Please be patient. Learning will take 5-20 minutes.


In [109]:
X_train[0][0]

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])

In [110]:
len(X_train_edge_features)

200

In [111]:
len(Y_train_flat)

200

In [112]:
X_train_directions[0][0][0, :]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [36]:
X_train_directions[0][1]

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 3,  4],
       [ 4,  5],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [14, 15],
       [15, 16],
       [16, 17],
       [18, 19],
       [19, 20],
       [20, 21],
       [21, 22],
       [22, 23],
       [24, 25],
       [25, 26],
       [26, 27],
       [27, 28],
       [28, 29],
       [30, 31],
       [31, 32],
       [32, 33],
       [33, 34],
       [34, 35],
       [36, 37],
       [37, 38],
       [38, 39],
       [39, 40],
       [40, 41],
       [ 0,  6],
       [ 1,  7],
       [ 2,  8],
       [ 3,  9],
       [ 4, 10],
       [ 5, 11],
       [ 6, 12],
       [ 7, 13],
       [ 8, 14],
       [ 9, 15],
       [10, 16],
       [11, 17],
       [12, 18],
       [13, 19],
       [14, 20],
       [15, 21],
       [16, 22],
       [17, 23],
       [18, 24],
       [19, 25],
       [20, 26],
       [21, 27],
       [22, 28],
       [23, 29

In [26]:
Y_train[0].shape

(7, 6)

In [21]:
Y_train_flat[0]

array([ 0,  0,  0,  0,  0,  0,  0,  2,  3,  0,  0,  0,  0,  1,  4,  5,  0,
        0,  0,  0,  0,  6,  7,  0,  0,  0,  0,  9,  8,  0,  0,  0,  0, 10,
        0,  0,  0,  0,  0,  0,  0,  0], dtype=uint8)

In [27]:
Y_train[0]

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  2,  3,  0,  0,  0],
       [ 0,  1,  4,  5,  0,  0],
       [ 0,  0,  0,  6,  7,  0],
       [ 0,  0,  0,  9,  8,  0],
       [ 0,  0,  0, 10,  0,  0],
       [ 0,  0,  0,  0,  0,  0]], dtype=uint8)

In [52]:
X_train[0].shape

(7, 6, 5)

In [51]:
X_train[0]

array([[[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0]],

       [[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0

In [55]:
X_train[0].shape

(7, 6, 5)

In [56]:
Y_train[0].shape

(7, 6)

In [70]:
X_train[0][0, 0]

array([0, 0, 0, 1, 0])

In [76]:
X_train_directions[0][0].shape

(42, 45)

In [82]:
X_train_edge_features[0][0].shape

(42, 45)

In [81]:
X_train_edge_features[0][1].shape

(71, 2)

In [88]:
X_train_directions[0][2]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

In [100]:
X_train_edge_features[0][2].shape

(71, 180)

In [87]:
Y_train[0][1, 1]

2

In [62]:
Y_train[0]

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  2,  3,  0,  0,  0],
       [ 0,  1,  4,  5,  0,  0],
       [ 0,  0,  0,  6,  7,  0],
       [ 0,  0,  0,  9,  8,  0],
       [ 0,  0,  0, 10,  0,  0],
       [ 0,  0,  0,  0,  0,  0]], dtype=uint8)

In [83]:
180/45

4.0

In [92]:
right, down = make_grid_edges(X_train[0], return_lists=True)

In [97]:
neighborhood_feature(X_train[0]).shape

(42, 45)

In [98]:
X_train[0]

(7, 6, 5)

In [99]:
right

array([[ 0,  1],
       [ 1,  2],
       [ 2,  3],
       [ 3,  4],
       [ 4,  5],
       [ 6,  7],
       [ 7,  8],
       [ 8,  9],
       [ 9, 10],
       [10, 11],
       [12, 13],
       [13, 14],
       [14, 15],
       [15, 16],
       [16, 17],
       [18, 19],
       [19, 20],
       [20, 21],
       [21, 22],
       [22, 23],
       [24, 25],
       [25, 26],
       [26, 27],
       [27, 28],
       [28, 29],
       [30, 31],
       [31, 32],
       [32, 33],
       [33, 34],
       [34, 35],
       [36, 37],
       [37, 38],
       [38, 39],
       [39, 40],
       [40, 41]])

In [101]:
edges = np.vstack([right, down])

In [102]:
edges.shape

(71, 2)

In [114]:
X_train_edge_features[0][2].shape

(71, 90)

In [115]:
X_train_edge_features[0][2][0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])